<a href="https://colab.research.google.com/github/peisuke/generative_ai_notebooks/blob/main/Audio/06_Style_Bert_VITS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/litagin02/Style-Bert-VITS2.git
%cd Style-Bert-VITS2

Cloning into 'Style-Bert-VITS2'...
remote: Enumerating objects: 3144, done.
remote: Counting objects: 100% (1515/1515), done.
remote: Compressing objects: 100% (390/390), done.
remote: Total 3144 (delta 1234), reused 1275 (delta 1123), pack-reused 1629
Receiving objects: 100% (3144/3144), 10.68 MiB | 14.61 MiB/s, done.
Resolving deltas: 100% (1947/1947), done.
/content/Style-Bert-VITS2


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 52.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 100.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.7/208.7 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [4]:
# Download voice data from https://zunko.jp/multimodal_dev/login.php
!cp /content/drive/MyDrive/colab_data/voice/ITA_emotion_normal_synchronized_wav.zip .
!unzip -q ITA_emotion_normal_synchronized_wav.zip

In [5]:
!mkdir -p Data/zundamon/raw
!mv ITA_emotion_normal_synchronized_wav/*.wav Data/zundamon/raw/

In [6]:
!wget https://raw.githubusercontent.com/mmorise/ita-corpus/main/emotion_transcript_utf8.txt -O emotion_transcript_utf8.txt

--2024-01-24 09:08:55--  https://raw.githubusercontent.com/mmorise/ita-corpus/main/emotion_transcript_utf8.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18899 (18K) [text/plain]
Saving to: ‘emotion_transcript_utf8.txt’

emotion_transcript_ 100%[===================>]  18.46K  --.-KB/s    in 0.001s  

2024-01-24 09:08:55 (21.8 MB/s) - ‘emotion_transcript_utf8.txt’ saved [18899/18899]



In [7]:
import re
import os
import glob

with open("emotion_transcript_utf8.txt", "r") as f:
    emotion_transcript = f.read()
    emotion_transcript = emotion_transcript.split("\n")
    emotion_transcript = list(filter(lambda x: len(x) > 0, emotion_transcript))
    emotion_transcript = [re.split(r'_|:|,', txt) for txt in emotion_transcript]

emotion_transcript_new = []
for et in emotion_transcript:
    emotion_transcript_new.append({
        "id": int(et[1]),
        "text": et[2]
    })

In [8]:
print(len(emotion_transcript_new))
print(emotion_transcript_new[0])

100
{'id': 1, 'text': 'えっ嘘でしょ。'}


In [9]:
filenames = sorted(glob.glob("Data/zundamon/raw/*.wav"))

filenames_new = []
for f in filenames:
    ret = re.match(r"emoNormal(\d{3}).wav", os.path.basename(f))
    if ret:
        filenames_new.append({
            "id": int(ret.group(1)),
            "filename": f
        })
filenames_new = {f["id"]: f["filename"] for f in filenames_new}

for et in emotion_transcript_new:
    id = et["id"]
    if id in filenames_new:
        et["filename"] = filenames_new[id]

In [10]:
print(len(emotion_transcript_new))
print(emotion_transcript_new[0])

100
{'id': 1, 'text': 'えっ嘘でしょ。', 'filename': 'Data/zundamon/raw/emoNormal001.wav'}


In [11]:
with open("Data/zundamon/esd.list", "w") as f:
    for et in emotion_transcript_new:
        filename = et["filename"].replace("raw", "wavs")
        line = filename + "|" + "zundamon" + "|" + "JP" + "|" + et["text"]
        f.writelines(line + "\n")

In [12]:
!python initialize.py

01-24 09:08:56 |  INFO  | initialize.py:16 | Downloading deberta-v2-large-japanese-char-wwm pytorch_model.bin
pytorch_model.bin: 100% 1.32G/1.32G [00:09<00:00, 134MB/s]
01-24 09:09:06 |  INFO  | initialize.py:16 | Downloading chinese-roberta-wwm-ext-large pytorch_model.bin
pytorch_model.bin: 100% 1.31G/1.31G [00:11<00:00, 117MB/s]
01-24 09:09:17 |  INFO  | initialize.py:16 | Downloading deberta-v3-large spm.model
spm.model: 100% 2.46M/2.46M [00:00<00:00, 206MB/s]
01-24 09:09:17 |  INFO  | initialize.py:16 | Downloading deberta-v3-large pytorch_model.bin
pytorch_model.bin: 100% 874M/874M [00:07<00:00, 117MB/s]
01-24 09:09:25 |  INFO  | initialize.py:30 | Downloading pretrained G_0.safetensors
G_0.safetensors: 100% 234M/234M [00:05<00:00, 42.4MB/s]
01-24 09:09:31 |  INFO  | initialize.py:30 | Downloading pretrained D_0.safetensors
D_0.safetensors: 100% 187M/187M [00:01<00:00, 180MB/s]
01-24 09:09:32 |  INFO  | initialize.py:30 | Downloading pretrained DUR_0.safetensors
DUR_0.safetensors:

In [13]:
import yaml
import json
import shutil

In [14]:
with open(os.path.join("configs", "paths.yml"), "r", encoding="utf-8") as f:
    path_config: dict[str, str] = yaml.safe_load(f.read())
    dataset_root = path_config["dataset_root"]
    # assets_root = path_config["assets_root"]

def get_path(model_name):
    assert model_name != "", "モデル名は空にできません"
    dataset_path = os.path.join(dataset_root, model_name)
    lbl_path = os.path.join(dataset_path, "esd.list")
    train_path = os.path.join(dataset_path, "train.list")
    val_path = os.path.join(dataset_path, "val.list")
    config_path = os.path.join(dataset_path, "config.json")
    return dataset_path, lbl_path, train_path, val_path, config_path

model_name = "zundamon"
batch_size = 4
epochs = 10
bf16_run = False
save_every_steps = 100

dataset_path, _, train_path, val_path, config_path = get_path(model_name)

if os.path.isfile(config_path):
    with open(config_path, "r", encoding="utf-8") as f:
        config = json.load(f)
else:
    # Use default config
    with open("configs/config.json", "r", encoding="utf-8") as f:
        config = json.load(f)

config["model_name"] = model_name
config["data"]["training_files"] = train_path
config["data"]["validation_files"] = val_path
config["train"]["batch_size"] = batch_size
config["train"]["epochs"] = epochs
config["train"]["bf16_run"] = bf16_run
config["train"]["eval_interval"] = save_every_steps

model_path = os.path.join(dataset_path, "models")
shutil.copytree(
    src="pretrained",
    dst=model_path,
)

with open(config_path, "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

In [15]:
if not os.path.exists("config.yml"):
    shutil.copy(src="default_config.yml", dst="config.yml")

with open("config.yml", "r", encoding="utf-8") as f:
    yml_data = yaml.safe_load(f)

yml_data["model_name"] = model_name
yml_data["dataset_path"] = dataset_path
with open("config.yml", "w", encoding="utf-8") as f:
    yaml.dump(yml_data, f, allow_unicode=True)

In [16]:
!python resample.py --in_dir Data/zundamon/raw \
    --out_dir Data/zundamon/wavs \
    --num_processes 1 \
    --sr 44100

100%|##########| 101/101 [00:11<00:00,  8.53it/s]
01-24 09:09:58 |  INFO  | resample.py:122 | Resampling Done!


In [17]:
!python preprocess_text.py --config-path Data/zundamon/config.json \
    --transcription-path Data/zundamon/esd.list \
    --train-path Data/zundamon/train.list \
    --val-path Data/zundamon/val.list

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
0it [00:00, ?it/s]Downloading: "https://github.com/r9y9/open_jtalk/releases/download/v1.11.1/open_jtalk_dic_utf_8-1.11.tar.gz"

dic.tar.gz: 0.00B [00:00, ?B/s]
dic.tar.gz:   0% 8.00k/22.6M [00:00<14:42, 26.8kB/s]
dic.tar.gz: 100% 22.6M/22.6M [00:00<00:00, 53.3MB/s]
Extracting tar file /usr/local/lib/python3.10/dist-packages/pyopenjtalk/dic.tar.gz
100it [00:01, 78.02it/s]
01-24 09:10:09 |  INFO  | preprocess_text.py:100 | Total repeated audios: 0, Total number of audio not foun

In [18]:
!python bert_gen.py --config Data/zundamon/config.json

100%|##########| 100/100 [00:24<00:00,  4.03it/s]
01-24 09:10:38 |  INFO  | bert_gen.py:85 | bert.pt is generated! total: 100 bert.pt files.


In [19]:
!python style_gen.py --config Data/zundamon/config.json --num_processes 1

2024-01-24 09:10:48.824705: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 09:10:48.824803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 09:10:48.938845: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-24 09:10:51.293681: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
pytorch_model.bin: 100% 26.6M/26.6M [00:00<00:00, 144MB/s]
config.yaml: 100% 221/221 [00:00<00:00, 1.60MB/s]
100%|##########| 100/100 [00:06<00:00, 15.35it/s]
01-24 09:11:01 |  INFO  | style_gen.py:77 | Finished generating style vectors! tot

In [20]:
!python train_ms.py --config Data/zundamon/config.json --model Data/zundamon

2024-01-24 09:11:05.213881: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 09:11:05.213947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 09:11:05.215302: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-24 09:11:05.222324: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-24 09:11:06.247062: W tensorflow/compiler/tf2

In [21]:
import re
import glob

In [22]:
model_path_list = glob.glob("model_assets/zundamon/*.safetensors")

In [23]:
display(model_path_list)

['model_assets/zundamon/zundamon_e8_s200.safetensors',
 'model_assets/zundamon/zundamon_e4_s100.safetensors',
 'model_assets/zundamon/zundamon_e10_s260.safetensors']

In [24]:
model_iter = []
for m in model_path_list:
    filename = os.path.basename(m)
    ret = re.match(r"^zundamon_e\d+_s(\d+).safetensors$", filename)
    iter = int(ret.group(1))
    model_iter.append({
        "iter": iter,
        "path": m
    })
model_path = sorted(model_iter, key=lambda x: x["iter"])[-1]["path"]

In [25]:
print(model_path)

model_assets/zundamon/zundamon_e10_s260.safetensors


In [26]:
from common.tts_model import Model
from text.japanese import g2kata_tone, kata_tone2phone_tone, text_normalize

In [27]:
model = Model(
    model_path=model_path,
    config_path="model_assets/zundamon/config.json",
    style_vec_path="model_assets/zundamon/style_vectors.npy",
    device="cuda",
)

In [28]:
line_split = True
kata_tone_json_str = ""

kata_tone = None
if kata_tone_json_str != "":
    assert line_split == False, "line_splitがTrueの場合、kata_tone_json_strは空にしてください"
    kata_tone = []
    json_data = json.loads(kata_tone_json_str)
    for kana, tone in json_data:
        assert isinstance(kana, str) and tone in (0, 1), f"{kana}, {tone}"
        kata_tone.append((kana, tone))

In [29]:
# toneは実際に音声合成に代入される際のみnot Noneになる
tone = None
if kata_tone is not None:
    phone_tone = kata_tone2phone_tone(kata_tone)
    tone = [t for _, t in phone_tone]

In [30]:
sr, audio = model.infer(
    text="こんにちは、初めまして。あなたの名前はなんていうの？ ",
    language="JP",
    reference_audio_path=None,
    sdp_ratio=0.2,
    noise=0.6,
    noisew=0.8,
    length=1,
    line_split=line_split,
    split_interval=0.5,
    assist_text="",
    assist_text_weight=1,
    use_assist_text=False,
    style="Neutral",
    style_weight=5,
    given_tone=tone,
    sid=0,
)

01-24 09:17:13 |  INFO  | tts_model.py:100 | Start generating audio data from text:
こんにちは、初めまして。あなたの名前はなんていうの？ 


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


01-24 09:17:15 |  INFO  | utils.py:168 | Loaded 'model_assets/zundamon/zundamon_e10_s260.safetensors' (iteration 10)
01-24 09:17:23 |  INFO  | tts_model.py:162 | Audio data generated successfully


In [31]:
from IPython.display import Audio
display(Audio(audio, rate=sr))